<a href="https://colab.research.google.com/github/it-math/analytica/blob/main/hh_request.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import time

In [ ]:
# Расширенный список ключевых слов для поиска IT-вакансий
SEARCH_TERMS = [
    # Программирование и разработка
    "программист", "разработчик", "developer", "backend", "frontend",
    "fullstack", "web разработчик", "mobile разработчик", "iOS разработчик",
    "Android разработчик", "C++", "Python", "Java", "JavaScript", "React",
    "Node.js", "Django", "Flask", "Vue", "Angular", "PHP", "Rust", "Go", "Haskell",
    "Prolog", "Lisp", "Swift", "Ruby", "C#", "Fortran", "Delphi", "R", "Matlab", "1C",
    "Wordpress", "Tilda", "Joomla", "Git", "GIT", "HTML", "CSS", "Kotlin", "SQL", "сеньор", "миддл",
    "стажер", "математик", "администратор информационных систем", "web-разработчик", "оператор ПК",


    # Тестирование
    "тестировщик", "QA", "quality assurance", "автотесты", "ручное тестирование",
    "Selenium", "Postman", "JMeter", "API тестирование",

    # Анализ данных и машинное обучение
    "data scientist", "аналитик", "machine learning", "ML", "исследователь данных", "NLP",
    "big data", "статистика", "Pandas", "NumPy", "TensorFlow", "PyTorch", "DS", "продуктовый аналитик",

    # DevOps и системное администрирование
    "DevOps", "системный администратор", "инженер devops", "Linux", "Docker",
    "Kubernetes", "CI/CD", "Ansible", "облачные технологии", "AWS", "Azure",

    # Безопасность
    "информация безопасность", "кибербезопасность", "pentester", "security engineer",
    "ethical hacker", "аудит безопасности",

    # Менеджмент и управление проектами
    "product manager", "менеджер продукта", "project manager", "руководитель проекта",
    "Agile", "Scrum", "Kanban", "Jira", "Project-manager", "IT", "ИС",

    # Поддержка и техническая документация
    "технический писатель", "support", "поддержка", "helpdesk", "специалист техподдержки",

    # Архитектура ПО
    "software architect", "архитектор", "solution architect", "проектирование систем"
]

In [ ]:
# Настройки
AREA = 113         # Россия
PER_PAGE = 100     # Вакансий на запрос
DELAY = 5          # Задержка между запросами
CSV_FILENAME = "it_vacancies_full.csv"

In [ ]:
def get_vacancies(keyword, area=AREA, per_page=PER_PAGE):
    url = "https://api.hh.ru/vacancies"
    params = {
        "text": keyword.strip(),
        "area": area,
        "per_page": per_page,
        "search_field": "name"
    }

    headers = {
        "User-Agent": "Mozilla/5.0",
        "HH-User-Agent": "MyVacancyAnalyzerApp/1.0"
    }

    print(f"Ищем: '{keyword}'")
    try:
        response = requests.get(url, params=params, headers=headers, timeout=10)
    except requests.exceptions.RequestException as e:
        print(f"Ошибка подключения при запросе '{keyword}': {e}")
        return pd.DataFrame()

    if response.status_code != 200:
        print(f"Ошибка для '{keyword}': {response.status_code}")
        print("Текст ответа:", response.text)
        return pd.DataFrame()

    data = response.json()
    items = data.get("items", [])

    if not items:
        print(f"Нет вакансий по запросу '{keyword}'")
        return pd.DataFrame()

    vacancies = []
    for item in items:
        salary = item.get("salary")
        salary_from = salary["from"] if salary and "from" in salary else None
        salary_to = salary["to"] if salary and "to" in salary else None

        key_skills = [skill["name"] for skill in item.get("key_skills", [])]
        prof_role = [role["name"] for role in item.get("professional_roles","")]
        d = item.get("snippet","")
        requirement = d["requirement"]
        if requirement:
            requirement = requirement.replace('<highlighttext>',' ')
            requirement = requirement.replace('</highlighttext>',' ')
        responsibility = d["responsibility"]
        if responsibility:
            responsibility = responsibility.replace('<highlighttext>',' ')
            responsibility = responsibility.replace('</highlighttext>',' ')
        vacancies.append({
            "name": item["name"],
            "company": item["employer"]["name"],
            "city": item["area"]["name"],
            "experience": item["experience"]["name"],
            "url": item["alternate_url"],
            "salary_from": salary_from,
            "salary_to": salary_to,
            "key_skills": ", ".join(key_skills),
            "search_term": keyword,
            "prof_role": prof_role,
            "requirement":requirement,
            "responsibility":responsibility

        })

    df = pd.DataFrame(vacancies)
    print(f"Найдено {len(df)} вакансий по запросу '{keyword}'")
    return df

In [ ]:
def collect_all_vacancies():
    all_vacancies = []

    for term in SEARCH_TERMS:
        df = get_vacancies(term, per_page=PER_PAGE)
        if not df.empty:
            all_vacancies.append(df)
        time.sleep(DELAY)

    if not all_vacancies:
        print("Не найдено ни одной вакансии.")
        return pd.DataFrame()

    full_df = pd.concat(all_vacancies, ignore_index=True)
    full_df.drop_duplicates(subset=["url"], keep="first", inplace=True)

    print(f"\n Всего уникальных вакансий: {len(full_df)}")
    full_df.to_csv(CSV_FILENAME, index=False, encoding="utf-8-sig")
    print(f" Все вакансии сохранены в файл: {CSV_FILENAME}")

    return full_df

In [ ]:
if __name__ == "__main__":
    print("Мастер-класс: Сбор и анализ IT-вакансий с hh.ru (через API)\n")
    df = collect_all_vacancies()

    if not df.empty:
        print("\n Пример первых 10 вакансий:")
        print(df[["name", "company", "city", "salary_from", "salary_to", "key_skills", "prof_role", "requirement","responsibility" ]].head())
    else:
        print("Ни одна вакансия не найдена.")

